In [42]:
import bs4
import requests

visitedLinks = set()

def get_links(start):
    links = set(start)
    res = requests.get(start)    
    soup = bs4.BeautifulSoup(res.text, 'lxml')    
    
    for link in soup.find_all('a', href=True):
        href = link["href"]
    
        # apenas links do site
        if (href.startswith(start) | href.startswith("/")):
        
            # trata urls relativos
            url = start + str(href)
            links.add(url)
            print(url) # teste
    
    # remove todos os links que sao iguais a buscas passadas
    links = links.difference(visitedLinks)
    
    # armazena todos os links já consultados sem repeticoes
    visitedLinks.update(links)
    
    print("visitedLinks: \n", visitedLinks) # teste
            
    return list(links)
    

def crawl_bsf(start): 
    get_links(start) # teste
#     queue = []
#     frontier = [start] # start -> frontier (frontier = get_links(start))
#     count = 0
    
#     while(len(frontier) != 0 and count < 100):  # mudar para 1000
#         seen = frontier.pop(0)        
#         queue.append(seen)
#         frontier.extend(get_links(seen)) # adiciona os vizinhos no fim da lista
#         count += 1
        
#     print(len(queue)) # quantas pages do site foram crawleadas
#     return queue

In [43]:
'''
seeds = ["https://www.minhavida.com.br", "https://medicoresponde.com.br",  "https://emagrecer.eco.br",
             "https://www.tuasaude.com", "https://www.mdsaude.com", "http://portalms.saude.gov.br", 
             "http://www.portalnatural.com", "http://portal.saude.pe.gov.br"]
'''

# descartados "https://minutosaudavel.com.br" e "http://www.riocomsaude.rj.gov.br"

# teste
seeds = ["https://www.minhavida.com.br"]

sites = []

for link in seeds:
    sites.append(crawl_bsf(link))  # lista de listas, para armazenar os links por site

https://www.minhavida.com.br/
https://www.minhavida.com.br/saude
https://www.minhavida.com.br/saude/temas
https://www.minhavida.com.br/saude/bulas
https://www.minhavida.com.br/guias/dor-nas-costas
https://www.minhavida.com.br/guias/saude-bucal
https://www.minhavida.com.br/guias/convivendo-com-artrite-psoriasica
https://www.minhavida.com.br/guias/dii
https://www.minhavida.com.br/guias/especial-medley
https://www.minhavida.com.br/guias/especial-autocuidado
https://www.minhavida.com.br/guias/diarreia
https://www.minhavida.com.br/guias/eu-de-lentes
https://www.minhavida.com.br/guias/so-entre-elas
https://www.minhavida.com.br/alimentacao
https://www.minhavida.com.br/alimentacao/tudo-sobre
https://www.minhavida.com.br/receita.htm
https://www.minhavida.com.br/guias/especial-vigor
https://www.minhavida.com.br/guias/o-poder-das-sementes
https://www.minhavida.com.br/beleza
https://www.minhavida.com.br/beleza/tudo-sobre
https://www.minhavida.com.br/fitness
https://www.minhavida.com.br/familia
htt

## Tarefas  

[x] crawlear  
[x] tratar links que não são do site  
[x] tratar links relativos   
[x] resgatar ordem de inserção em sets (trata links que são iguais) 
[x] BFS  
[ ] tratar links que são imagens  
[ ] sleep - não sobrecarregar  
[ ] respeitar robots.txt 
[ ] score para classificação no crawler